# SVF.PropDbReader — Interactive Examples

This notebook demonstrates all features of the **SVF.PropDbReader** library using a .NET Interactive kernel.

## Prerequisites

- [.NET 8.0 SDK](https://dotnet.microsoft.com/download/dotnet/8.0)
- [.NET Interactive / Polyglot Notebooks](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode)
- An Autodesk APS access token and a model URN (for cloud examples), OR a local `.sdb` file

## 🔧 Setup — Install the NuGet Package

In [ ]:
#r "nuget: SVF.PropDbReader, 1.1.0.0"
#r "nuget: Microsoft.Data.Sqlite, 8.0.0"

## 📝 Configuration — Enter Your Variables

Fill in **one** of the two options below:

**Option A**: Use a local `.sdb` file  
**Option B**: Download from Autodesk APS using access token + URN

In [ ]:
// ============================================================
// ⚙️ CONFIGURATION — Fill in your values below
// ============================================================

// Option A: Local .sdb file path
string localDbPath = @"C:\path\to\your\properties.sdb";  // <-- CHANGE THIS

// Option B: Autodesk APS credentials (leave empty if using Option A)
string accessToken = "";  // <-- Your APS access token
string modelUrn = "";     // <-- Your model URN

// Choose which option to use
bool useLocalFile = true;  // Set to false to use APS download

Console.WriteLine($"Mode: {(useLocalFile ? "Local File" : "APS Download")}");

## 🚀 Create the Reader

In [ ]:
using SVF.PropDbReader;

PropDbReader reader;

if (useLocalFile)
{
    reader = new PropDbReader(localDbPath);
    Console.WriteLine($"✅ Opened local database: {localDbPath}");
}
else
{
    reader = await PropDbReader.CreateAsync(accessToken, modelUrn);
    Console.WriteLine($"✅ Downloaded and opened database for URN: {modelUrn}");
}

---
## 📁 Schema Discovery
Discover what categories and properties exist in the model.

### Get All Categories

In [ ]:
var categories = await reader.GetAllCategoriesAsync();

Console.WriteLine($"Found {categories.Count} categories:\n");
foreach (var cat in categories)
{
    Console.WriteLine($"  📁 {cat}");
}

### Get All Property Names

In [ ]:
var propertyNames = await reader.GetAllPropertyNamesAsync();

Console.WriteLine($"Found {propertyNames.Count} unique property names:\n");
foreach (var name in propertyNames)
{
    Console.WriteLine($"  🏷️ {name}");
}

### Get Categories with Their Properties (Full Schema Tree)

In [ ]:
var schema = await reader.GetCategoriesWithPropertiesAsync();

foreach (var (category, properties) in schema)
{
    Console.WriteLine($"\n📁 {category} ({properties.Count} properties)");
    foreach (var propName in properties)
    {
        Console.WriteLine($"   ├─ {propName}");
    }
}

---
## 🔍 Single Element Queries

### Get Direct Properties for an Element

In [ ]:
long dbId = 1;  // <-- CHANGE THIS to a valid dbId in your model

var props = await reader.GetPropertiesForDbIdAsync(dbId);

Console.WriteLine($"Element dbId={dbId} has {props.Count} properties:\n");
foreach (var kvp in props.OrderBy(k => k.Key))
{
    Console.WriteLine($"  {kvp.Key} = {kvp.Value}");
}

### Get Merged (Inherited) Properties

In [ ]:
var merged = await reader.GetMergedPropertiesAsync(dbId);

Console.WriteLine($"Element dbId={dbId} — merged properties ({merged.Count} total):\n");
foreach (var kvp in merged.OrderBy(k => k.Key))
{
    Console.WriteLine($"  {kvp.Key} = {kvp.Value}");
}

### Get a Single Property Value

In [ ]:
string category = "Dimensions";     // <-- CHANGE as needed
string displayName = "Width";       // <-- CHANGE as needed

var value = await reader.GetPropertyValueAsync(dbId, category, displayName);

Console.WriteLine(value != null
    ? $"dbId {dbId}: {category}.{displayName} = {value}"
    : $"dbId {dbId}: {category}.{displayName} is not set");

### Get Parent Element

In [ ]:
var parentId = await reader.GetParentDbIdAsync(dbId);

if (parentId.HasValue)
{
    Console.WriteLine($"Parent of dbId {dbId} is dbId {parentId.Value}");
    var parentProps = await reader.GetPropertiesForDbIdAsync(parentId.Value);
    Console.WriteLine($"  Parent has {parentProps.Count} properties");
}
else
{
    Console.WriteLine($"Element dbId {dbId} has no parent.");
}

---
## 📊 Bulk Property Queries

### Get All Values for a Property (Dictionary)

In [ ]:
string bulkCategory = "Identity Data";  // <-- CHANGE as needed
string bulkProperty = "Name";           // <-- CHANGE as needed

var allValues = await reader.GetAllPropertyValuesAsync(bulkCategory, bulkProperty);

Console.WriteLine($"{bulkCategory}.{bulkProperty} — found {allValues.Count} elements:\n");
foreach (var kvp in allValues.Take(20))  // Show first 20
{
    Console.WriteLine($"  dbId {kvp.Key}: {kvp.Value}");
}
if (allValues.Count > 20)
    Console.WriteLine($"  ... and {allValues.Count - 20} more");

### Stream Property Values (Memory-Efficient)

In [ ]:
int count = 0;
await foreach (var (id, val) in reader.GetAllPropertyValuesStreamAsync(bulkCategory, bulkProperty))
{
    if (count < 10)  // Print first 10
        Console.WriteLine($"  dbId {id}: {val}");
    count++;
}
Console.WriteLine($"\nStreamed {count} total values.");

### Get as List of Tuples (LINQ-Friendly)

In [ ]:
var list = await reader.GetAllPropertyValuesListAsync(bulkCategory, bulkProperty);

// Example: group by value and count
var grouped = list
    .Where(x => x.value != null)
    .GroupBy(x => x.value!.ToString()!)
    .OrderByDescending(g => g.Count())
    .Take(10);

Console.WriteLine($"Top 10 most common {bulkProperty} values:\n");
foreach (var group in grouped)
{
    Console.WriteLine($"  {group.Key}: {group.Count()} elements");
}

### Thread-Safe ConcurrentDictionary

In [ ]:
using System.Collections.Concurrent;

var concurrent = await reader.GetAllPropertyValuesConcurrentAsync(bulkCategory, bulkProperty);
Console.WriteLine($"ConcurrentDictionary has {concurrent.Count} entries.");

### Stream into a ConcurrentDictionary

In [ ]:
var dict = new ConcurrentDictionary<long, object?>();
await reader.GetAllPropertyValuesStreamToConcurrentAsync(bulkCategory, bulkProperty, dict);
Console.WriteLine($"Streamed {dict.Count} entries into ConcurrentDictionary.");

---
## 🌐 Full Database Queries

### Get All Properties for All Elements

In [ ]:
// ⚠️ WARNING: This loads everything into memory. Use only for small/medium models.
var allProps = await reader.GetAllPropertiesAsync();

Console.WriteLine($"Total elements: {allProps.Count}\n");

// Show first 3 elements
foreach (var (id, properties) in allProps.Take(3))
{
    Console.WriteLine($"--- Element dbId={id} ({properties.Count} properties) ---");
    foreach (var (key, val) in properties.Take(5))
    {
        Console.WriteLine($"  {key} = {val}");
    }
    if (properties.Count > 5)
        Console.WriteLine($"  ... and {properties.Count - 5} more");
}

### Stream All Properties (Memory-Efficient)

In [ ]:
int streamCount = 0;
await foreach (var (id, key, val) in reader.GetAllPropertiesStreamAsync())
{
    if (streamCount < 10)
        Console.WriteLine($"  dbId {id}: {key} = {val}");
    streamCount++;
}
Console.WriteLine($"\nStreamed {streamCount} total property rows.");

### Find Elements by Property Value

In [ ]:
string searchCategory = "__category__";  // <-- CHANGE as needed
string searchDisplayName = "";            // <-- CHANGE as needed
string searchValue = "Walls";             // <-- CHANGE as needed

var foundIds = await reader.FindDbIdsByPropertyAsync(searchCategory, searchDisplayName, searchValue);

Console.WriteLine($"Found {foundIds.Count} elements where {searchCategory}.{searchDisplayName} = '{searchValue}'\n");
foreach (var id in foundIds.Take(20))
{
    Console.WriteLine($"  dbId: {id}");
}

---
## 🔧 Custom SQL Queries

### Simple Query

In [ ]:
var countResult = await reader.QueryAsync("SELECT COUNT(*) AS total_elements FROM _objects_id");
Console.WriteLine($"Total elements in database: {countResult[0]["total_elements"]}");

### Parameterized Query (Recommended)

In [ ]:
var paramResults = await reader.QueryAsync(
    "SELECT * FROM _objects_attr WHERE category = $cat ORDER BY display_name",
    new Dictionary<string, object?> { ["$cat"] = "Dimensions" }
);

Console.WriteLine($"Attributes in 'Dimensions' category:\n");
foreach (var row in paramResults)
{
    Console.WriteLine($"  id={row["id"]}, display_name={row["display_name"]}");
}

### Count Elements per Category

In [ ]:
var categoryCounts = await reader.QueryAsync(@"
    SELECT _objects_attr.category, COUNT(DISTINCT _objects_eav.entity_id) AS element_count
    FROM _objects_eav
    INNER JOIN _objects_attr ON _objects_eav.attribute_id = _objects_attr.id
    WHERE _objects_attr.category IS NOT NULL AND _objects_attr.category != ''
    GROUP BY _objects_attr.category
    ORDER BY element_count DESC
");

Console.WriteLine("Elements per category:\n");
foreach (var row in categoryCounts)
{
    Console.WriteLine($"  {row["category"]}: {row["element_count"]} elements");
}

---
## 📍 Fragment Locations (New in v1.1.0)

Access 3D element locations without downloading hundreds of MB of fragment geometry.
All location data is stored on disk in the `.sdb` SQLite database — **zero memory overhead**.

### Embed Locations into Database (Disk-Only)

In [ ]:
// Embed locations into the .sdb file (all on disk — zero memory overhead)
// NOTE: This disposes the previous reader and creates one with embedded locations
if (useLocalFile)
{
    // If the local file already has embedded locations, nothing to do
    if (reader.HasFragmentLocations)
    {
        Console.WriteLine($"✅ Database already has {reader.FragmentLocationCount} embedded locations");
    }
    else
    {
        Console.WriteLine("⚠️ Location embedding requires APS credentials. Set useLocalFile=false and provide credentials.");
    }
}
else
{
    reader.Dispose();
    reader = await PropDbReader.CreateWithEmbeddedLocationsAsync(accessToken, modelUrn);
    Console.WriteLine($"✅ Embedded {reader.FragmentLocationCount} fragment locations into {reader.DbPath}");
}

### Get Properties with Location

In [ ]:
if (!reader.HasFragmentLocations)
{
    Console.WriteLine("⚠️ No embedded locations. Run the cell above first.");
}
else
{
    long locationDbId = 1;  // <-- CHANGE THIS
    
    var (properties, location) = await reader.GetPropertiesWithLocationAsync(locationDbId);
    
    Console.WriteLine($"Element {locationDbId}:");
    Console.WriteLine($"  Position: ({location.X:F2}, {location.Y:F2}, {location.Z:F2})");
    Console.WriteLine($"  BBox Min: ({location.MinX:F2}, {location.MinY:F2}, {location.MinZ:F2})");
    Console.WriteLine($"  BBox Max: ({location.MaxX:F2}, {location.MaxY:F2}, {location.MaxZ:F2})");
    Console.WriteLine($"  Size: {(location.MaxX - location.MinX):F2} × {(location.MaxY - location.MinY):F2} × {(location.MaxZ - location.MinZ):F2}");
    Console.WriteLine($"\n  Properties ({properties.Count}):");
    foreach (var prop in properties.Take(5))
    {
        Console.WriteLine($"    {prop.Key} = {prop.Value}");
    }
}

### Find Elements by Property with Locations

In [ ]:
if (!reader.HasFragmentLocations)
{
    Console.WriteLine("⚠️ No embedded locations.");
}
else
{
    string filterCategory = "__category__";  // <-- CHANGE as needed
    string filterProperty = "";
    string filterValue = "Walls";  // <-- CHANGE as needed
    
    var results = await reader.FindByPropertyWithLocationsAsync(filterCategory, filterProperty, filterValue);
    
    Console.WriteLine($"Found {results.Count} elements where {filterCategory}.{filterProperty} = '{filterValue}'\n");
    
    foreach (var (id, props, loc) in results.Take(10))
    {
        Console.WriteLine($"dbId {id}:");
        Console.WriteLine($"  Position: ({loc.X:F2}, {loc.Y:F2}, {loc.Z:F2})");
        Console.WriteLine($"  Height: {(loc.MaxZ - loc.MinZ):F2}");
    }
    
    if (results.Count > 10)
        Console.WriteLine($"... and {results.Count - 10} more");
}

### Stream All Properties with Locations

In [ ]:
if (!reader.HasFragmentLocations)
{
    Console.WriteLine("⚠️ No embedded locations.");
}
else
{
    int locationCount = 0;
    
    // Streams one row at a time from SQLite — constant memory usage
    await foreach (var (id, props, loc) in reader.GetAllPropertiesWithLocationsStreamAsync())
    {
        if (locationCount < 10)
        {
            Console.WriteLine($"dbId {id}: Position ({loc.X:F2}, {loc.Y:F2}, {loc.Z:F2}), {props.Count} properties");
        }
        locationCount++;
    }
    
    Console.WriteLine($"\nStreamed {locationCount} elements with locations (all from disk).");
}

### Embed Locations into Existing Database

In [ ]:
// Alternative: embed locations into an existing .sdb file
// Useful if you downloaded the DB first and want to add locations later
if (reader.HasFragmentLocations)
{
    Console.WriteLine($"✅ Database already has {reader.FragmentLocationCount} embedded locations.");
    Console.WriteLine($"   File: {reader.DbPath}");
}
else if (!useLocalFile)
{
    Console.WriteLine("Embedding locations into existing database...");
    await reader.EmbedFragmentLocationsAsync(accessToken, modelUrn);
    Console.WriteLine($"✅ Embedded {reader.FragmentLocationCount} locations into {reader.DbPath}");
    Console.WriteLine("Next time you open this file, locations are ready — no re-download needed!");
}
else
{
    Console.WriteLine("⚠️ Embedding requires APS credentials.");
}

### Query Single Location from Database

In [ ]:
// Query a single location directly from the SQLite database on disk
if (reader.HasFragmentLocations)
{
    int queryDbId = 1;  // <-- CHANGE THIS
    var location = await reader.GetFragmentLocationAsync(queryDbId);
    
    if (location.HasValue)
    {
        Console.WriteLine($"Location for dbId {queryDbId}:");
        Console.WriteLine($"  ({location.Value.X:F2}, {location.Value.Y:F2}, {location.Value.Z:F2})");
    }
    else
    {
        Console.WriteLine($"No location found for dbId {queryDbId}");
    }
}
else
{
    Console.WriteLine("⚠️ No embedded locations. Run the embedding cell above first.");
}

### Batch Query with Locations

In [ ]:
if (!reader.HasFragmentLocations)
{
    Console.WriteLine("⚠️ No embedded locations.");
}
else
{
    int[] batchIds = { 1, 2, 3, 4, 5 };  // <-- CHANGE THIS
    
    var batch = await reader.GetPropertiesWithLocationsBatchAsync(batchIds);
    
    Console.WriteLine($"Retrieved {batch.Count} elements:\n");
    foreach (var (id, props, loc) in batch)
    {
        Console.WriteLine($"dbId {id}: ({loc.X:F2}, {loc.Y:F2}, {loc.Z:F2}), {props.Count} properties");
    }
}

### Compare Fragment Download Sizes

Traditional fragment download vs location-only download:

In [ ]:
// Shows the efficiency of disk-based location storage

if (reader.HasFragmentLocations)
{
    int elementCount = reader.FragmentLocationCount;
    long locationBytes = elementCount * 36;  // 9 floats × 4 bytes each
    
    Console.WriteLine($"Element count: {elementCount:N0}");
    Console.WriteLine($"Location data on disk: {locationBytes:N0} bytes ({locationBytes / 1024.0 / 1024.0:F2} MB)");
    Console.WriteLine($"Location data in memory: 0 bytes (all queries go through SQLite)");
    Console.WriteLine($"\n❌ Full fragment download: ~100-500 MB (includes geometry, materials, etc.)");
    Console.WriteLine($"✅ Location-only (embedded in .sdb): ~{locationBytes / 1024.0 / 1024.0:F2} MB on disk, 0 MB in memory");
    Console.WriteLine($"\nDownload reduction: ~{100.0 * (1.0 - (locationBytes / (200.0 * 1024 * 1024))):F0}% smaller");
}

---
## 🧹 Cleanup

In [ ]:
reader.Dispose();
Console.WriteLine("✅ Reader disposed and resources released.");